In [4]:
import os
import cv2
import numpy as np
import pandas as pd
import threading
from queue import Queue
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

datasetNAME = 'SIPI'
if not os.path.exists('coverdata.csv'):
    pd.DataFrame(columns=['METRIC','PSNR','SSIM','CC','DATASET']).to_csv('coverdata.csv', index=False)

In [5]:
def toprint(df):
    columns_to_analyze = df.columns[1:]  # Start from the 2nd column (Column2)
    max_values = df[columns_to_analyze].max()
    min_values = df[columns_to_analyze].min()
    mean_values = df[columns_to_analyze].mean()
    
    # Create a new DataFrame with max, min, and mean values
    output_df = pd.DataFrame({
        'Metric': ['Max', 'Min', 'Mean'],
        'PSNR': [max_values['PSNR'], min_values['PSNR'], mean_values['PSNR']],
        'SSIM': [max_values['SSIM'], min_values['SSIM'], mean_values['SSIM']],
        'CC': [max_values['CC'], min_values['CC'], mean_values['CC']]
    })
    
    # Add a new column 'Dataset' with value 'newdata01'
    output_df['Dataset'] = datasetNAME
    output_df.to_csv('coverdata.csv', mode='a', header=False, index=False)

    return output_df

In [6]:
%%time


def read_file(path):
    img = cv2.imread(path)
    return img

def calculate_metrics(image_path, embedded_path, queue):
    original_image = read_file(image_path)
    embedded_image = read_file(embedded_path)
    
    psnr_value = psnr(original_image, embedded_image)
    ssim_value = ssim(original_image, embedded_image, channel_axis=-1)
    correlation = np.corrcoef(original_image.flatten(), embedded_image.flatten())[0, 1]
    
    queue.put((image_path, psnr_value, ssim_value, correlation))
    
    return psnr_value, ssim_value, correlation



def get_image_names(folder_path):
    image_names = []
    for filename in os.listdir(folder_path):
        image_names.append(filename)
    return image_names


images_DIR = "imagespng\\"
embedded_DIR = "embedded\\"
image_names = get_image_names(images_DIR)
data = {'Image Name': [], 'PSNR': [], 'SSIM': [], 'CC': []}
queue = Queue() 
threads = [] 


for imgname in image_names:
    image_path = images_DIR + imgname
    embedded_path = embedded_DIR + 'watermarked_image_' + imgname
    
    # create a thread for each image pair and pass the queue as an argument
    thread = threading.Thread(target=calculate_metrics, args=(image_path, embedded_path, queue))
    threads.append(thread) # add the thread to the list
    thread.start() # start the thread

for thread in threads:
    thread.join() # wait for all threads to finish

while not queue.empty(): # get the results from the queue
    image_path, psnr_value, ssim_value, correlation = queue.get()
    
    data['Image Name'].append(os.path.basename(image_path))
    data['PSNR'].append(psnr_value)
    data['SSIM'].append(ssim_value)
    data['CC'].append(correlation)
    


df = pd.DataFrame(data)





CPU times: total: 11.3 s
Wall time: 1.18 s


In [6]:
df.to_csv('filecover.csv')

In [8]:
toprint(df)

,Metric,PSNR,SSIM,CC,Dataset
0,Max,40.862977,0.983252,0.999381,SIPI
1,Min,37.422307,0.956894,0.995527,SIPI
2,Mean,39.297493,0.973974,0.997915,SIPI
